<a href="https://colab.research.google.com/github/zaq9/case/blob/master/case20190110_sq_result.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

#最終清算値市場データ取得（１月限）　
df = pd.read_csv(
    'https://raw.githubusercontent.com/zaq9/case/master/case20190110/data20190110.csv',index_col=0)

df

,PRICE
C20000,180
C20125,100
C20250,48
C20375,19
C20500,6
C20625,3
C20750,1
P19500,4
P19625,7
P19750,10


In [2]:
#colabo 用インストール
!pip install pulp

    100% |████████████████████████████████| 13.6MB 2.5MB/s 
  Running setup.py bdist_wheel for pulp ... - \ | / - \ | done
  Stored in directory: /root/.cache/pip/wheels/3b/f1/f5/f6d0aac56dd154c2593d3fa5e901aa9bc32a51bd8594b4bf8e
Successfully built pulp


In [0]:
import pulp as pu


class Payoff():
    def __init__(self, name):
        self.type = name[0]
        self.strike = int(name[1:])
    def __call__(self, u):
        v = u - self.strike
        if(self.type =='C'): 
            return v if v>0 else 0
        else:
            return -v if v<0 else 0

In [4]:
v_sq = 20290    #　SQ値
v_fut = 20125   #　mini先物価格
max_cost = 100  # 最大初期コスト
maxPayoff = 250 # 最大支払い額
max_num = 10    # 最大オプション枚数
max_num_fut = 100 #最大mini先物枚数
max_sum_sell = 10 #最大売りオプション枚数

df["buy"] = df.index.map(lambda x: pu.LpVariable(x, 0, max_num, pu.LpInteger))
df["sell"] = df.index.map(lambda x: pu.LpVariable(x+'_s', 0,max_num, pu.LpInteger))
df["payoff"] = df.index.map(lambda x: Payoff(x))

fut=  pu.LpVariable('fut', -max_num_fut, max_num_fut, pu.LpInteger)

def pay(s, objs): return pu.lpSum(pu.lpDot(objs, [p(s) for p in df.payoff]))
def v(s): 
    return pay(s, df['buy']) - pay(
        s, df['sell']) +fut*0.1*(s-v_fut) 

cost = pu.lpSum(pu.lpDot(df['buy'], df['PRICE'])) -pu.lpSum(pu.lpDot(df['sell'], df['PRICE']))
m = pu.LpProblem('Max', sense=pu.LpMaximize)
m += v(v_sq ) -cost
m += cost <= max_cost
m += pu.lpSum(df['sell']) <= max_sum_sell
# 合計売り枚数

for sq in range(18000,23000,125): m += v(sq) >= -maxPayoff
    
m.solve()
df['ans'] = df.buy.map(lambda x: x.value()) - df.sell.map(lambda x: x.value())

print(f"制約条件1：最大初期コスト={max_cost}：最大支払額＝{maxPayoff}")
print(f"制約条件2：最大売り合計枚数={max_sum_sell:.0f}：最大権利価格別枚数＝{max_num}")
print(f"制約条件3：最大mini先物枚数={max_num_fut:.0f}：mini先物初期値＝{v_fut}")

print()
print(f"最大収益 ={m.objective.value():.0f}(@SQ={v_sq})：初期コスト={sum(df.ans * df.PRICE):.0f}")
print(f"mini[{fut.value():.0f}]@{v_fut} >>{v_sq}" )
for x in df.query( ' ans != 0 ').itertuples():
    print(f"{x.Index}[{x.ans:.0f}]@{x.PRICE}  >> {x.payoff(v_sq)}")

制約条件1：最大初期コスト=100：最大支払額＝250
制約条件2：最大売り合計枚数=10：最大権利価格別枚数＝10
制約条件3：最大mini先物枚数=100：mini先物初期値＝20125

最大収益 =664(@SQ=20290)：初期コスト=86
mini[70]@20125 >>20290
C20250[-8]@48  >> 40
C20375[-1]@19  >> 0
C20750[2]@1  >> 0
P20125[8]@94  >> 0
P20375[-1]@265  >> 85
